In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from iris import irisRequests
import ngl
import datetime
from numba import njit
from numba.typed import Dict
from sklearn.metrics.pairwise import haversine_distances

In [2]:
@njit
def construct_aftershocks_map_steps(region, coords, dlat, dlon, steps):
    N_lat = int((region[1]-region[0])/dlat)
    N_lon = int((region[3]-region[2])/dlon)
    shocks_map = np.zeros((N_lat, N_lon))
    for coord in coords:
        i,j = int((coord[0]-region[0])/dlat), int((coord[1]-region[2])/dlon)
        if(steps > 0):
            for di in range(-steps,steps+1):
                for dj in range(-steps, steps+1):
                    if(i+di >= 0 and i+di < N_lat and j+dj >= 0 and j+dj < N_lon):
                        shocks_map[i+di,j+dj] += 1
            #for di in range(-steps,0):
            #    if(i+di >= 0):
            #        shocks_map[i+di,j] += 1
            #for di in range(1,steps+1):
            #    if(i+di < N_lat):
            #        shocks_map[i+di,j] += 1
            #for dj in range(-steps,0):
            #    if(j+dj >= 0):
            #        shocks_map[i,j+dj] += 1
            #for dj in range(1,steps+1):
            #    if(j+dj < N_lon):
            #        shocks_map[i,j+dj] += 1
        else:
            shocks_map[i,j] += 1
    return shocks_map

In [3]:
@njit
def construct_aftershocks_map(region, coords, dlat, dlon):
    N_lat = int((region[1]-region[0])/dlat)
    N_lon = int((region[3]-region[2])/dlon)
    shocks_map = np.zeros((N_lat, N_lon))
    for coord in coords:
        i,j = int((coord[0]-region[0])/dlat), int((coord[1]-region[2])/dlon)
        shocks_map[i,j] += 1
        #if(i>0):
        #    shocks_map[i-1,j] += 1
        #if(j>0):
        #    shocks_map[i,j-1] += 1
        #if(i < N_lat-1):
        #    shocks_map[i+1,j] += 1
        #if(j < N_lon-1):
        #    shocks_map[i,j+1] += 1
    return shocks_map

In [4]:
rootpath = "csv_24/"

In [5]:
regions = {}
regions['greece'] = (30, 45,18, 44)
regions['california'] = (30, 41, -125, -113)
regions['japan'] = (20, 50, 120, 150)
regions['italy'] = (35,46,6, 19)
regions_m = {}
regions_m['japan'] = (6.5, 4.0)
regions_m['italy'] = (5.0, 3.0)
regions_m['greece'] = (6.0, 4.0)
regions_m['california'] = (5.0, 3.0)

In [6]:
station_list = ngl.ngl_process_list(ngl.ngl_24h_2w) # daily measurements, with 2 weeks delay

In [7]:
station_info = {}
for name, region in regions.items():
    station_names, station_lats, station_lons  = ngl.get_all_stations_box(station_list, *region)
    station_info[name]= (station_names, station_lats, station_lons)

In [8]:
station_data = {}
for name, s_info in station_info.items():
    for s_cnt, s_name in enumerate(s_info[0]):
        df, status = ngl.ngl_retrieve_24h(rootpath, s_name)
        print(s_name, status)
        station_data[s_name] = df

ABEL downloaded
ADAN downloaded
ADCS downloaded
ADIY downloaded
ADN1 downloaded
ADN2 downloaded
ADY1 downloaded
AFY0 downloaded
AFYN downloaded
AFYT downloaded
AGID downloaded
AGIG downloaded
AGIO downloaded
AGNA downloaded
AGRI downloaded
AIGI downloaded
AKD1 downloaded
AKDG downloaded
AKHI downloaded
AKHR downloaded
AKHU downloaded
AKLE downloaded
AKS1 downloaded
AKSR downloaded
AKYR downloaded
ALE3 downloaded
ALNY downloaded
ALON downloaded
ALX2 downloaded
ALXR downloaded
AMA3 downloaded
AMAS downloaded
AMFI downloaded
AMMN downloaded
ANAV downloaded
ANDR downloaded
ANIK downloaded
ANK2 downloaded
ANKR downloaded
ANKY downloaded
ANMU downloaded
ANOC downloaded
ANOP downloaded
ANRK downloaded
ANTA downloaded
ANTL downloaded
ANTP downloaded
APK1 downloaded
ARDA downloaded
AREL downloaded
ARG2 downloaded
ARK1 downloaded
ARKI downloaded
ARPK downloaded
ARSA downloaded
ARST downloaded
ART1 downloaded
ART2 downloaded
ARTV downloaded
ASGA downloaded
ASSO downloaded
AST5 downloaded
ASTY dow

KeyboardInterrupt: 

In [ ]:
start_time = datetime.datetime(2003, 1, 1, 0, 0, 0)
end_time =  datetime.datetime(2024, 1, 1, 0, 0, 0)
catalogs = {}
for name, region in regions.items():
    download_url =irisRequests.url_events_box(start_time, end_time, region[0], region[1], region[2], region[3], minmag=3, magtype="MW")
    df = pd.read_csv(download_url, sep="|", comment="#")
    df.Time = pd.to_datetime(df.Time, errors='coerce')
    df.dropna(axis=0, inplace=True)
    df.sort_values(by="Time", inplace=True)
    df.reset_index(inplace=True, drop=True)
    catalogs[name] = df

In [ ]:
@njit(nogil=True)
def construct_map(u, idx, d, d_cutoff = 0.01):
    # u = (N_s, 3)
    # idx = (N_s)
    # d = (N_lat, N_lon, N_s_tot)
    u_map = np.zeros((d.shape[0], d.shape[1], 3))
    for i in range(0, d.shape[0]):
        for j in range(0, d.shape[1]):
            cnt = 0
            for i_n,n in enumerate(idx):
                d_ijn = d[i,j,n]
                if(d_ijn <= d_cutoff):
                    u_map[i,j,:] = u_map[i,j,:] + u[i_n, :]
                    cnt += 1
            if(cnt > 0):
                u_map[i,j,:] = u_map[i,j,:]/float(cnt)
            else:
                u_map[i,j,0] = np.nan
                u_map[i,j,1] = np.nan
                u_map[i,j,2] = np.nan
    return u_map


In [ ]:
maximal_time_shift = np.timedelta64(7, 'D')
minimal_time_shift = np.timedelta64(1, 'D')
dlat = 0.1
dlon = 0.1
datasets = {}
for name, catalog in catalogs.items():
    dataset = []
    print(name)
    # discretization of the region
    region = regions[name]
    N_lat = int((region[1]-region[0])/dlat)
    N_lon = int((region[3]-region[2])/dlon)
    grid_latlat, grid_lonlon = np.meshgrid( region[0] + np.arange(0, N_lat)*dlat, region[2] + np.arange(0, N_lon)*dlon, indexing='ij')
    grid_latlat = grid_latlat.flatten()
    grid_lonlon = grid_lonlon.flatten()
    grid = np.hstack([grid_latlat[:,None], grid_lonlon[:,None]])
    stations_coords = np.hstack([station_info[name][1][:,None], station_info[name][2][:,None]])
    grid_stations_dists = haversine_distances(np.radians(grid), np.radians(stations_coords))
    grid_stations_dists = grid_stations_dists.reshape((N_lat, N_lon, -1))
    

    # earthquakes identification
    large_shocks = catalog[catalog.Magnitude >= regions_m[name][0]]
    large_shocks_days = large_shocks.Time
    large_shocks_coords =  large_shocks[['Latitude','Longitude']].values
    for day_time, coord in zip(large_shocks_days, large_shocks_coords):
        print(day_time)
        day = np.datetime64(datetime.datetime(day_time.year, day_time.month, day_time.day))
        aftershocks = catalog[ (catalog.Magnitude >= regions_m[name][1])* (catalog.Time.values.astype('datetime64[D]') >=  day + minimal_time_shift)*(catalog.Time.values.astype('datetime64[D]') <= pd.to_datetime(day) +maximal_time_shift ) ]
        aftershocks_coords = aftershocks[['Latitude','Longitude']].values
        #aftershocks_map = construct_aftershocks_map(region, aftershocks_coords, dlat, dlon)
        aftershocks_maps = [construct_aftershocks_map_steps(region, aftershocks_coords, dlat, dlon, steps) for steps in [0,1,2] ]
        # daily data
        data = []
        indices = []
        for cnt, (s_name, _,__) in  enumerate(zip(*station_info[name])):
            df = station_data[s_name]
            row = df[df.date==  day]
            row_p = df[df.date==  day - np.timedelta64(1, 'D')]
            if(len(row) > 0 and len(row_p) > 0):
                u_n = row.lat.values[0]-row_p.lat.values[0]
                u_e = row.lon.values[0]-row_p.lon.values[0]
                u_v = row.height.values[0] - row_p.height.values[0]
                data.append((u_n, u_e, u_v))
                indices.append(cnt)
        if(len(indices) > 0):
            data = np.array(data)
            indices = np.array(indices)
            
            u_map = construct_map(data, indices, grid_stations_dists)
            dataset.append((day_time, coord,u_map, aftershocks_maps))
            #print(data)
            fig, ax = plt.subplots(ncols=4, figsize=(16,3))
            ax[0].contourf(np.sqrt(u_map[:,:,0]**2+u_map[:,:,1]**2), cmap='hsv')
            for am_c, am in enumerate(aftershocks_maps):
                ax[am_c+1].contourf(am)
            #locs = np.argwhere(aftershocks_map>0)
            #ax[0].contour(aftershocks_map, cmap='Greys', levels=2)
            #ax.scatter(locs[:,1], locs[:,0], color='black', s=5)
            plt.show()
            print(len(indices))
    datasets[name] = dataset

In [ ]:
study_name = "japan"
dataset = datasets[study_name]
x_train = []
x_test = []
y_train = []
y_test = []
limit_date = datetime.datetime(2015,1,1,0,0,0, tzinfo=datetime.timezone.utc)
for data in dataset:
    if(data[0]  > limit_date):
        y_test.append(data[3][-2])
        x_test.append(data[2])
    else:
        y_train.append(data[3][-2])
        x_train.append(data[2])
y_train = np.array(y_train)
y_test = np.array(y_test)
x_train = np.array(x_train)
x_test = np.array(x_test)
np.savez(study_name + ".npz", x_train=x_train, x_test=x_test, y_train = y_train, y_test=y_test)